# groupby 공부 두번째
pandas로 dplyr처럼 그룹바이에 익숙해지기 위한 공부  
데이터 전처리의 엄청나게 많은 부분은 그룹바이를 잘 쓰는 것으로 해결된다고 생각한다.  
그룹바이를 자유자재로 다루자.  

https://rfriend.tistory.com/391?category=675917

# 3. groupby 처리 메서드와 함수

pandas에서 groupby 적용을 할 때 (1) pandas에 내장되어 있는 기술 통계량 메서드를 사용하는 방법과, (2) (사용자정의) 함수를 agg(function) 형태로 사용할 수 있다. 기존 메서드를 사용하는 것이 성능이 최적화되어 있기 때문에 함수를 사용하는 것 보다 빠르다. 메서드가 지원하는 부분에 대해서는 메서드를 사용하고 그 이외의 것들에 대해서는 사용자정의 함수를 사용하자. 메서드들은 NA는 모두 무시하는 형태로 워킹한다.

In [69]:
import numpy as np
import pandas as pd

In [70]:
df = pd.DataFrame({'group':['a', 'a', 'a', 'b', 'b', 'b'],
                'value_1':np.arange(6),
                'value_2':np.random.randn(6)})
df

,group,value_1,value_2
0,a,0,1.025875
1,a,1,-0.173349
2,a,2,1.424930
3,b,3,2.337311
4,b,4,1.028397
5,b,5,1.431399


In [71]:
grouped = df.groupby('group')
grouped

## (1) groupby 메서드를 이용한 계산

### 1-1. count(), sum()

In [72]:
grouped.count()


,value_1,value_2
group,,
a,3,3
b,3,3


In [73]:
grouped.sum()

,value_1,value_2
group,,
a,3,2.277456
b,12,4.797108


In [74]:
pd.DataFrame(grouped.sum()['value_2'])

,value_2
group,
a,2.277456
b,4.797108


### 1-2. min(), max()

In [75]:
grouped.min()
grouped.max()

,value_1,value_2
group,,
a,2,1.424930
b,5,2.337311


### 1-3. mean(), median

In [76]:
grouped.mean()
grouped.median()

,value_1,value_2
group,,
a,1.0,1.025875
b,4.0,1.431399


In [77]:
grouped.mean().add_prefix('mean_')

,mean_value_1,mean_value_2
group,,
a,1.0,0.759152
b,4.0,1.599036


변수의 결과값에 접두사를 붙이는 방법

### 1-4. std(), var(), quantile()

분산과 표준편차 계산에는 n-1의 자유도를 사용한다.

### 1-5. first(), last()

### 1-6. describe()

In [78]:
grouped.describe()['value_1']

,count,mean,std,min,25%,50%,75%,max
group,,,,,,,,
a,3.0,1.0,1.0,0.0,0.5,1.0,1.5,2.0
b,3.0,4.0,1.0,3.0,3.5,4.0,4.5,5.0


## (2) 함수를 이용한 groupby 집계 : agg(function)

IQR 계산을 통한 예시

In [79]:
df

,group,value_1,value_2
0,a,0,1.025875
1,a,1,-0.173349
2,a,2,1.424930
3,b,3,2.337311
4,b,4,1.028397
5,b,5,1.431399


In [80]:
def iqr_func(x):
    q3, q1 = np.percentile(x, [75,25])
    iqr = q3 - q1
    return iqr

In [81]:
grouped.agg(iqr_func)

,value_1,value_2
group,,
a,1.0,0.799140
b,1.0,0.654457


In [82]:
grouped.quantile([0.75, 0.25])

value_1   value_2
group                        
a     0.75      1.5  1.225402
      0.25      0.5  0.426263
b     0.75      4.5  1.884355
      0.25      3.5  1.229898

# 4. groupb.agg 의 다양한 방법

- (1) 함수 list를 사용하여 다수의 groupby 집계 함수를 통일한 컬럼에 적용하기
- (2) 컬럼과 함수를 매핑한 dict를 사용하여 컬럼별로 특정 grouopby집계 함수를 적용하기
- (3) (이름, 함수)의 튜플을 사용하여 groupby 집계 함수에 이름 부여하기

In [83]:
import numpy as np
import pandas as pd
import csv
import urllib3

url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data'
dat = pd.read_csv(url,
                      names = ['sex', 'length', 'diameter', 'height', 
                               'whole_weight', 'shucked_weight', 'viscera_weight', 
                               'shell_weight', 'rings'], 
                      header = None)
dat.head()

,sex,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [84]:
dat['length_cat'] = np.where(dat.length > np.median(dat.length), 'length_long', 'length_short')
dat[['length','length_cat']][:10]

,length,length_cat
0,0.455,length_short
1,0.350,length_short
2,0.530,length_short
3,0.440,length_short
4,0.330,length_short
5,0.425,length_short
6,0.530,length_short
7,0.545,length_short
8,0.475,length_short
9,0.550,length_long


## (1) 함수 list를 사용하여 다수의 groupby 집계 함수를 동일한 컬럼에 적용하기

grouped.agg() 안에 사용될 함수를 캐릭터형태로 넣는 것도 가능하다.

In [85]:
grouped_ww = dat.groupby(['sex', 'length_cat'])['whole_weight']
print(grouped_ww.mean())
print(grouped_ww.agg('mean'))

sex  length_cat  
F    length_long     1.261330
     length_short    0.589702
I    length_long     0.923215
     length_short    0.351234
M    length_long     1.255182
     length_short    0.538157
Name: whole_weight, dtype: float64
sex  length_cat  
F    length_long     1.261330
     length_short    0.589702
I    length_long     0.923215
     length_short    0.351234
M    length_long     1.255182
     length_short    0.538157
Name: whole_weight, dtype: float64


In [86]:
grouped_ww.agg(['size', 'mean', 'std', 'min', 'max'])

size      mean       std     min     max
sex length_cat                                            
F   length_long    889  1.261330  0.329656  0.6405  2.6570
    length_short   418  0.589702  0.202400  0.0800  1.3580
I   length_long    188  0.923215  0.218334  0.5585  2.0495
    length_short  1154  0.351234  0.204237  0.0020  1.0835
M   length_long    966  1.255182  0.354682  0.5990  2.8255
    length_short   562  0.538157  0.246498  0.0155  1.2825

In [87]:
function_list = ['size', 'mean', 'std', 'min', 'max']
grouped_ww.agg(function_list)

size      mean       std     min     max
sex length_cat                                            
F   length_long    889  1.261330  0.329656  0.6405  2.6570
    length_short   418  0.589702  0.202400  0.0800  1.3580
I   length_long    188  0.923215  0.218334  0.5585  2.0495
    length_short  1154  0.351234  0.204237  0.0020  1.0835
M   length_long    966  1.255182  0.354682  0.5990  2.8255
    length_short   562  0.538157  0.246498  0.0155  1.2825

In [88]:
grouped = dat.groupby(['sex', 'length_cat'])
function_list = ['size', 'mean', 'std']
result = grouped[['whole_weight','shell_weight']].agg(function_list)
result['whole_weight'][['size','mean']]

size      mean
sex length_cat                  
F   length_long    889  1.261330
    length_short   418  0.589702
I   length_long    188  0.923215
    length_short  1154  0.351234
M   length_long    966  1.255182
    length_short   562  0.538157

groupby 결과 데이터프레임으로 부터 row를 기준으로 indexing 할 수도 있다. row기준으로 indexing할 때 df.loc 하는 것과 같다.

In [89]:
result.loc['M']

whole_weight                     shell_weight                    
                     size      mean       std         size      mean       std
length_cat                                                                    
length_long           966  1.255182  0.354682          966  0.351683  0.102636
length_short          562  0.538157  0.246498          562  0.162141  0.075629

In [90]:
result.loc['M', 'shell_weight']

,size,mean,std
length_cat,,,
length_long,966,0.351683,0.102636
length_short,562,0.162141,0.075629


## (2) 컬럼과 함수를 매핑한 dictionary를 사용하여 컬럼별로 특정 groupby 함수를 적용하기

사용자 정의 함수를 2개 정의한 후에 grouped.agg()를 통해 적용해보자

In [91]:
def range_func(x):
    max_val = np.max(x)
    min_val = np.min(x)
    range_val = max_val - min_val
    return range_val

def iqr_func(x):
    q3, q1 = np.percentile(x, [75, 25])
    iqr = q3 - q1
    return iqr

dictionary를 활용해서 'whole_weight'에는 size(), mean(), std() 메서드를 사용하여 매핑하여 groupby를 적용하고 'shell_weight'에는 사용자정의 함수를 적용해보자.
이거 잘쓰면 복잡한것들을 간단하게 표현할 수도 있겠다.

In [92]:
d = {'whole_weight': ['size', 'mean', 'std'],
    'shell_weight': [range_func, iqr_func]}
grouped.agg(d)

whole_weight                     shell_weight          
                         size      mean       std   range_func  iqr_func
sex length_cat                                                          
F   length_long           889  1.261330  0.329656        0.850  0.127000
    length_short          418  0.589702  0.202400        0.378  0.080500
I   length_long           188  0.923215  0.218334        0.485  0.067875
    length_short         1154  0.351234  0.204237        0.349  0.092750
M   length_long           966  1.255182  0.354682        0.776  0.124000
    length_short          562  0.538157  0.246498        0.375  0.102750

## (3) (이름, 함수)의 튜플을 사용하여 groupby 함수에 이름 부여하기

In [93]:
d = {'whole_weight': ['size', 'mean', 'std'],
    'shell_weight': [('Range', range_func), ('Inter-Quartile_Range',iqr_func)]}
grouped.agg(d)

whole_weight                     shell_weight  \
                         size      mean       std        Range   
sex length_cat                                                   
F   length_long           889  1.261330  0.329656        0.850   
    length_short          418  0.589702  0.202400        0.378   
I   length_long           188  0.923215  0.218334        0.485   
    length_short         1154  0.351234  0.204237        0.349   
M   length_long           966  1.255182  0.354682        0.776   
    length_short          562  0.538157  0.246498        0.375   

                                       
                 Inter-Quartile_Range  
sex length_cat                         
F   length_long              0.127000  
    length_short             0.080500  
I   length_long              0.067875  
    length_short             0.092750  
M   length_long              0.124000  
    length_short             0.102750

# 5. 여러개의 컬럼에 대해 다른 함수를 적용한 groupby : grouped.apply(functions)

- (1) 데이터프레임에서 여러개의 컬럼에 대해 다른 함수 적용하여 groupby : grouped.apply(functions)
- (2) 계층적 인덱스를 가진 데이터프레임의 여러개의 컬럼에 대해 다른 함수 적용 : grouped(level=['index1','index2']).apply(function)

## (1) 데이터프레임에서 여러개의 컬럼에 대해 다른 함수 적용 : grouped.apply(functions)

In [94]:
import numpy as np
import pandas as pd

df = pd.DataFrame({'grp_col_1' : ['a', 'a', 'a', 'a', 'a', 'b', 'b', 'b', 'b', 'b'], 
                   'grp_col_2' : ['c', 'c', 'd', 'd', 'd', 'e', 'e', 'f', 'f', 'f'], 
                   'val_1' : np.arange(10),                   
                   'val_2' : np.random.randn(10)})

df

,grp_col_1,grp_col_2,val_1,val_2
0,a,c,0,-0.243019
1,a,c,1,-0.344006
2,a,d,2,-2.753905
3,a,d,3,0.277944
4,a,d,4,-0.296688
5,b,e,5,-0.677616
6,b,e,6,0.007516
7,b,f,7,0.857043
8,b,f,8,-0.506933
9,b,f,9,-0.727028


In [95]:
def func(x):
    d = {}
    d['val_1_mean'] = x['val_1'].mean()
    d['val_2_std'] = x['val_1'].std()
    d['val_2_max'] = x['val_2'].max()
    d['val_2_min'] = x['val_2'].min()
    d['val_2_range'] = x['val_2'].max() - x['val_2'].min()
    return pd.Series(d, index=['val_1_mean', 'val_1_std', 'val_2_max', 'val_2_min', 'val_2_range'])

df_return = df.groupby(['grp_col_1', 'grp_col_2']).apply(func)
df_return


val_1_mean  val_1_std  val_2_max  val_2_min  val_2_range
grp_col_1 grp_col_2                                                          
a         c                 0.5        NaN  -0.243019  -0.344006     0.100987
          d                 3.0        NaN   0.277944  -2.753905     3.031849
b         e                 5.5        NaN   0.007516  -0.677616     0.685132
          f                 8.0        NaN   0.857043  -0.727028     1.584071

In [96]:
df_return.reset_index()

,grp_col_1,grp_col_2,val_1_mean,val_1_std,val_2_max,val_2_min,val_2_range
0,a,c,0.5,NaN,-0.243019,-0.344006,0.100987
1,a,d,3.0,NaN,0.277944,-2.753905,3.031849
2,b,e,5.5,NaN,0.007516,-0.677616,0.685132
3,b,f,8.0,NaN,0.857043,-0.727028,1.584071


In [97]:
df_return.reset_index(level='grp_col_2')

,grp_col_2,val_1_mean,val_1_std,val_2_max,val_2_min,val_2_range
grp_col_1,,,,,,
a,c,0.5,NaN,-0.243019,-0.344006,0.100987
a,d,3.0,NaN,0.277944,-2.753905,3.031849
b,e,5.5,NaN,0.007516,-0.677616,0.685132
b,f,8.0,NaN,0.857043,-0.727028,1.584071


## (2) 계층적 인덱스를 가진 데이터프레임의 여러개의 컬럼에 대해 다른 함수 적용하여 groupby 집계하기

In [98]:
arrays = [['a', 'a', 'a', 'a', 'a', 'b', 'b', 'b', 'b', 'b'], 
         ['c', 'c', 'd', 'd', 'd', 'e', 'e', 'f', 'f', 'f']]

myindex  = pd.MultiIndex.from_arrays(arrays, names=('grp_idx_1','grp_idx_2'))

df2 = pd.DataFrame({'val_1':np.arange(10),
            'val_2':np.random.randn(10)},
            index = myindex)

In [99]:
df2.groupby(level=['grp_idx_1', 'grp_idx_2']).apply(func)

val_1_mean  val_1_std  val_2_max  val_2_min  val_2_range
grp_idx_1 grp_idx_2                                                          
a         c                 0.5        NaN   0.454301  -0.126921     0.581222
          d                 3.0        NaN  -0.058367  -2.131692     2.073325
b         e                 5.5        NaN   0.213566  -1.466345     1.679911
          f                 8.0        NaN   1.794919  -0.010754     1.805673

# 6. 데이터 프레임 유지한 상태로 groupby하기 (R로 했을 때 groupby + mutate)

In [117]:
import numpy as np
import pandas as pd

df = pd.DataFrame({'group_1': ['a', 'a', 'a', 'a', 'a',  
                                    'b', 'b', 'b', 'b', 'b',], 
                    'group_2': ['c', 'c', 'c', 'd', 'd', 
                                    'e', 'e', 'e', 'f', 'f'], 
                    'col': [1, 2, np.NaN, 4, np.NaN, 
                            6, 7, np.NaN, 9, 10]})
df

,group_1,group_2,col
0,a,c,1.0
1,a,c,2.0
2,a,c,NaN
3,a,d,4.0
4,a,d,NaN
5,b,e,6.0
6,b,e,7.0
7,b,e,NaN
8,b,f,9.0
9,b,f,10.0


## (1) 그룹별 NaN이 아닌 원소 개수 구하여 데이터프레임에 새로운 칼럼 추가하기

In [118]:
df['count_col'] = df.groupby(['group_1', 'group_2'])['col'].transform('count')
df

,group_1,group_2,col,count_col
0,a,c,1.0,2
1,a,c,2.0,2
2,a,c,NaN,2
3,a,d,4.0,1
4,a,d,NaN,1
5,b,e,6.0,2
6,b,e,7.0,2
7,b,e,NaN,2
8,b,f,9.0,2
9,b,f,10.0,2


사용자정의 함수로 transform 사용해보기

In [119]:
def func(x):
    return len(x) +999
df['count_col'] = df.groupby(['group_1', 'group_2'])['col'].transform(func)
df

,group_1,group_2,col,count_col
0,a,c,1.0,1002
1,a,c,2.0,1002
2,a,c,NaN,1002
3,a,d,4.0,1001
4,a,d,NaN,1001
5,b,e,6.0,1002
6,b,e,7.0,1002
7,b,e,NaN,1002
8,b,f,9.0,1001
9,b,f,10.0,1001
